In [1]:
import numpy as np
from numpy import linalg as LA
import pickle
import sys
import os
from keras.models import load_model
from scipy.linalg import eigh

def cosine(x):
    d_inner = np.dot(x.T,x) #inner product

    l2 = np.linalg.norm(x,axis=0)
    d_norm = np.outer(l2.T,l2) #l2 norm
    d_norm = (d_inner+0.00001)/(d_norm+0.00001)
    return d_norm 

import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
session = tf.Session(config=config)

fcn1_initial = load_model('relu_cifar10_5000-2500-2500-1250/initial.h5')
fcn1_final = load_model('relu_cifar10_5000-2500-2500-1250'+'/final.h5')
print(1)
fcn2_initial = load_model('relu_cifar10_10000-1000-1000-1000/initial.h5')
fcn2_final = load_model('relu_cifar10_10000-1000-1000-1000'+'/final.h5')
print(2)
vgg11_initial = load_model('vgg11_cifar10/initial.h5')
vgg11_final = load_model('vgg11_cifar10'+'/final.h5')
print(3)
vgg16_initial = load_model('vgg16_cifar10/initial.h5')
vgg16_final = load_model('vgg16_cifar10'+'/final.h5')
print(4)
vgg19_initial = load_model('vgg19_cifar10/initial.h5')
vgg19_final = load_model('vgg19_cifar10'+'/final.h5')
print(5)
resnet18_initial = load_model('resnet18_cifar10/initial.h5')
resnet18_final = load_model('resnet18_cifar10'+'/final.h5')
print(6)
resnet34_initial = load_model('resnet34_cifar10/initial.h5')
resnet34_final = load_model('resnet34_cifar10'+'/final.h5')
print(7)
resnet50_initial = load_model('resnet50_cifar10/initial.h5')
resnet50_final = load_model('resnet50_cifar10'+'/final.h5')
print(8)
densenet121_initial = load_model('densenet121_cifar10/initial.h5')
densenet121_final = load_model('densenet121_cifar10'+'/final.h5')
print(9)

Using TensorFlow backend.
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/LOCAL/gjin/anaconda3/lib/python3.7/site-packages/tensorflo

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
1
2
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
3
4
5
6
7
8
9


In [3]:
import sys 
sys.path.append("..") 
import utils

gap = []
trn, tst = utils.get_cifar10('FCN')
for z in [fcn1_final, fcn2_final]:
    trnloss, trnacc = z.evaluate(trn.X, trn.Y, verbose=0)
    tstloss, tstacc = z.evaluate(tst.X, tst.Y, verbose=0)
    print(tstloss-trnloss)
    gap.append(tstloss-trnloss)

trn, tst = utils.get_cifar10('CNN')    
for z in [vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]:
    trnloss, trnacc = z.evaluate(trn.X, trn.Y, verbose=0)
    tstloss, tstacc = z.evaluate(tst.X, tst.Y, verbose=0)
    print(tstloss-trnloss)
    gap.append(tstloss-trnloss)

print(gap)

2.0565349135599287
2.3547526305638256
0.9291898392868041
0.5536880187988282
0.6781374630737305
2.681919717798233
2.552002730770111
2.807675155150108
1.437852966171616
[2.0565349135599287, 2.3547526305638256, 0.9291898392868041, 0.5536880187988282, 0.6781374630737305, 2.681919717798233, 2.552002730770111, 2.807675155150108, 1.437852966171616]


In [4]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('PFN:')

PFN_measure = []
for z in all_final:
    wf = z.get_weights()
    pp = 1
    for w in wf:
        if len(w.shape)>1:
            pp = pp*LA.norm(w) 
    PFN_measure.append(pp)
print(PFN_measure)

PFN:
[81164634.96651596, 33338642.146074183, 85778043279.79686, 5083848646973853.0, 1.1301702231164035e+19, 2.529652002844311e+22, 9.960333480014198e+34, 1.4008034169039389e+76, 5.958950833828354e+176]


In [5]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('PSN:')

PSN_measure = []
for z in all_final:
    pp = 1
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            if wf[i].shape[0] > wf[i].shape[1]:
                aa = wf[i].T.dot(wf[i])
            else:
                aa = wf[i].dot(wf[i].T)     
            pp = pp * np.sqrt(np.max(np.sum(np.abs(aa),axis=1)))
        elif len(wf[i].shape) == 4:
            aa = 0
            for w1 in wf[i]:
                for w2 in w1:
                    zz = w2.dot(w2.T)
                    aa = aa + np.max(np.sum(np.abs(zz),axis=1))
            pp = pp * np.sqrt(aa)
    PSN_measure.append(pp)
print(PSN_measure)    

PSN:
[14619.510795730364, 8558.538992292668, 143497.5849207305, 13999415.60210112, 294699100.0001145, 1054712421890.5027, 4.951817310863388e+16, 7.54292594178186e+46, 1.4065152575886064e+151]


In [6]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('NOP:')

NOP_measure = []
for z in all_final:
    pp = 1
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            pp = pp + wf[i].shape[0]*wf[i].shape[1]
        elif len(wf[i].shape) == 4:
            pp = pp + wf[i].shape[0]*wf[i].shape[1]*wf[i].shape[2]*wf[i].shape[3]
    NOP_measure.append(pp)
print(NOP_measure)  

NOP:
[37247501, 42730001, 9747137, 15239873, 20548289, 11176129, 21276865, 23475393, 6880449]


In [7]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]
all_initial = [fcn1_initial, fcn2_initial,vgg11_initial, vgg16_initial, vgg19_initial, resnet18_initial, resnet34_initial, resnet50_initial, densenet121_initial] 
print('SOSP:')

SOSP_measure = []
for z,x in zip(all_final,all_initial):
    n=0
    pp=0
    wf = z.get_weights()
    w0 = x.get_weights()
    
    for i in range(len(wf)):
        wf[i]=wf[i]-w0[i]
        if len(wf[i].shape) == 2:
            n = n + wf[i].shape[0]*wf[i].shape[1]
        elif len(wf[i].shape) == 4:
            n = n + wf[i].shape[0]*wf[i].shape[1]*wf[i].shape[2]*wf[i].shape[3]

        if len(wf[i].shape) == 2:
            if wf[i].shape[0] > wf[i].shape[1]:
                aa = wf[i].T.dot(wf[i])
            else:
                aa = wf[i].dot(wf[i].T)     
            pp = pp + np.sqrt(np.max(np.sum(np.abs(aa),axis=1)))
        elif len(wf[i].shape) == 4:
            aa = 0
            for w1 in wf[i]:
                for w2 in w1:
                    zz = w2.dot(w2.T)
                    aa = aa + np.max(np.sum(np.abs(zz),axis=1))
            pp = pp + np.sqrt(aa)
    pp = n*pp
    SOSP_measure.append(pp)
print(SOSP_measure) 

SOSP:
[1600035538.7222767, 1606762606.716156, 248887595.94726652, 527655564.3831213, 812336935.0760801, 845250733.9371884, 3125297496.0727882, 6105026277.913242, 15671366949.111446]


In [8]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('WC:')

WC_measure = []
for z in all_final:
    pp = 1
    n=0
    counter = 0
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            n = n + np.mean(np.abs(cosine(wf[i][:9])))
            counter += 1
        elif len(wf[i].shape) == 4 and wf[i].shape[0]>1:
            wf[i] = wf[i].transpose((2,3,0,1))
            wf[i] = wf[i].reshape(wf[i].shape[0],wf[i].shape[1],-1)
            for zz in wf[i]:
                n = n + np.mean(np.abs(cosine(zz.T)))
                counter += 1
    WC_measure.append(n/counter)
print(WC_measure)   

WC:
[0.29769465923309324, 0.2963906764984131, 0.2737462690443411, 0.275206018666095, 0.2740141661072216, 0.732712105421176, 0.7338545935068093, 0.2787850319944992, 0.3578975387298746]


In [9]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final]
all_initial = [fcn1_initial, fcn2_initial,vgg11_initial, vgg16_initial, vgg19_initial, resnet18_initial, resnet34_initial, resnet50_initial, densenet121_initial] 
print('PB:')

PB_measure = []
for z,x in zip(all_final,all_initial):
    pp=0
    wf = z.get_weights()
    w0 = x.get_weights()
    
    for i in range(len(wf)):
        pp = pp+LA.norm(wf[i]-w0[i])**2
    PB_measure.append(pp/2)
print(PB_measure)

PB:
[11636.039458743406, 8857.855593098007, 2022.3866813273316, 2623.5754268042674, 3345.768628566619, 4763.476229957958, 10353.829066383485, 16221039.808389993, 1049850584.7461755]


In [11]:
all_final = [fcn1_final, fcn2_final,vgg11_final, vgg16_final, vgg19_final, resnet18_final, resnet34_final, resnet50_final, densenet121_final] 
print('gw:')

gw_measure = []
for z in all_final:
    counter = 0
    n=0
    wf = z.get_weights()
    for i in range(len(wf)):
        if len(wf[i].shape) == 2:
            zz = np.mean(np.abs(cosine(wf[i])))
            n = n - (wf[i].shape[1]-1)*wf[i].shape[0]*np.log(1-zz)-wf[i].shape[0]*np.log(1+wf[i].shape[1]*zz-zz)
            counter += 1
        elif len(wf[i].shape) == 4 and wf[i].shape[0]>1:
            n_in = wf[i].shape[2]
            n_out = wf[i].shape[3]

            wf[i] = wf[i].transpose((2,3,0,1))
            wf[i] = wf[i].reshape(wf[i].shape[0],wf[i].shape[1],-1)
            zz = 0
            for xx in wf[i]:
                zz = zz + np.mean(np.abs(cosine(xx.T)))/len(wf[i])
            if zz < 1:    
                zzz = (n_out-1)*n_in*np.log(1-zz)-n_in*np.log(1+n_out*zz-zz)
            else:
                zzz = -50000
            n = n - zzz
            counter += 1
    gw_measure.append(n/counter)
print(gw_measure) 

gw:
[102539.96375062593, 115985.68734740582, 32171.97327675808, 34698.972006557546, 39308.05430474721, 129849.96658356296, 120448.80603449888, 23776.65004808116, 2034.134567273592]


In [12]:
import numpy as np
def kendall(a,b):
    pos = 0
    neg = 0
    for i in range(len(a)-1):
        for j in range(i+1,len(a)):
            if (a[i]-a[j])*(b[i]-b[j])>0:
                pos += 1
            else:
                neg += 1
    print('POS:',pos)
    print('NEG:',neg)
    print((pos-neg)/(pos+neg))
    
print('-----PFN-----')    
kendall(gap,np.array(PFN_measure)) 

print('-----PSN-----')  
kendall(gap,np.array(PSN_measure)) 

print('-----NOP-----')  
kendall(gap,np.array(NOP_measure)) 

print('-----SOSP-----')  
kendall(gap,np.array(SOSP_measure)) 

print('-----WC-----')  
kendall(gap,np.array(WC_measure)) 

print('-----PB-----')  
kendall(gap,np.array(PB_measure)) 

print('-----gw-----') 
kendall(gap,np.array(gw_measure)) 

print('-----PBC-----') 
kendall(gap,np.array(PB_measure)+np.array(gw_measure)) 

-----PFN-----
POS: 21
NEG: 15
0.16666666666666666
-----PSN-----
POS: 21
NEG: 15
0.16666666666666666
-----NOP-----
POS: 22
NEG: 14
0.2222222222222222
-----SOSP-----
POS: 26
NEG: 10
0.4444444444444444
-----WC-----
POS: 24
NEG: 12
0.3333333333333333
-----PB-----
POS: 24
NEG: 12
0.3333333333333333
-----gw-----
POS: 24
NEG: 12
0.3333333333333333
-----PBC-----
POS: 29
NEG: 7
0.6111111111111112
